In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

In [3]:
df = pd.read_csv("data.csv")
df

,age,affordability,have_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age', 'affordability']], df.have_insurance, test_size=0.2, random_state=25)

In [9]:
x_train_scaled = x_train.copy()
x_train_scaled['age']=x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age']=x_test_scaled['age']/100

In [10]:
print(len(x_test_scaled))
print(len(y_test))
print(len(x_train_scaled))
print(len(y_train))

6
6
22
22


In [11]:
x_train_scaled

,age,affordability
0,0.22,1
13,0.20,0
6,0.55,0
17,0.64,0
24,0.46,0
19,0.22,1
25,0.27,1
16,0.45,1
20,0.33,1
3,0.52,0


In [13]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

In [14]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics='accuracy')


In [ ]:
model.fit(x_train_scaled, y_train, epochs=5000)

Epoch 1/5000
1/1 [==============================] - 0s 368ms/step - loss: 0.7572 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7568 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7564 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7561 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7557 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7554 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.7550 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7547 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7543 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7540 - accuracy: 0.5000
Epoch 11/5000
1/1

In [16]:
x_test_scaled.shape

(6, 2)

In [17]:
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 0s 94ms/step - loss: 0.4228 - accuracy: 1.0000


[0.42278429865837097, 1.0]

In [18]:
x_test_scaled.shape

(6, 2)

In [19]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 50ms/step


array([[0.5994593 ],
       [0.22329907],
       [0.61318326],
       [0.33678332],
       [0.54380244],
       [0.768462  ]], dtype=float32)

In [20]:
y_test

2     1
10    0
21    1
11    0
14    1
9     1
Name: have_insurance, dtype: int64

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [22]:
coef, intercept = model.get_weights()
coef, intercept

(array([[ 5.688814e+00],
        [-6.122124e-04]], dtype=float32),
 array([-2.2699177], dtype=float32))

In [23]:
intercept
coef[0]
coef[1]

array([-0.00061221], dtype=float32)

In [24]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [25]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age+coef[1]*affordibility+intercept
    return sigmoid(weighted_sum)

In [26]:
prediction_function(.47,1)

0.5994592638311123

In [27]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [28]:
epsilon=1e-15

In [29]:
def log_loss(y_real, y_predicted):
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_real*np.log(y_predicted_new)+(1-y_real)*np.log(1-y_predicted_new))

In [31]:
def gradient_descent(age, affordability, y_true, epochs):
    w1 = w2 = 10
    bias = 10
    rate = 0.1
    n = len(age)
    
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        
        loss = log_loss(y_true, y_predicted)
        
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
        
        bias_d = np.mean(y_predicted-y_true)
        
        w1 = w1-rate*w1d
        w2 = w2-rate*w2d
        
        bias = bias-rate*bias_d
        
        print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias: {bias}, loss:{loss}")
    
    return w1, w2, bias

In [33]:
gradient_descent(x_train_scaled['age'], x_train_scaled['affordability'], y_train,1000)

Epoch:0, w1:9.98509093652149, w2:9.977272727277189, bias: 9.95000010757002, loss:8.763637399080597
Epoch:1, w1:9.970181874569118, w2:9.954545454554731, bias: 9.900000221086165, loss:8.73124938720235
Epoch:2, w1:9.95527281422786, w2:9.931818181832657, bias: 9.850000340877276, loss:8.698861455449029
Epoch:3, w1:9.940363755587425, w2:9.909090909110995, bias: 9.800000467290385, loss:8.666473472430848
Epoch:4, w1:9.925454698742524, w2:9.88636363638978, bias: 9.750000600691727, loss:8.634085465861874
Epoch:5, w1:9.910545643793144, w2:9.863636363669045, bias: 9.700000741467806, loss:8.601697530127854
Epoch:6, w1:9.895636590844845, w2:9.840909090948829, bias: 9.650000890026504, loss:8.569309646823447
Epoch:7, w1:9.88072754000907, w2:9.818181818229174, bias: 9.600001046798285, loss:8.536921732864064
Epoch:8, w1:9.865818491403477, w2:9.795454545510124, bias: 9.550001212237424, loss:8.50453375491146
Epoch:9, w1:9.850909445152276, w2:9.772727272791727, bias: 9.50000138682334, loss:8.47214589938841

(7.365393975731652, 0.20303738612626732, -3.0464324539813172)

# Rewrite

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

In [35]:
df = pd.read_csv('data.csv')
df

,age,affordability,have_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


In [36]:
from sklearn.model_selection import train_test_split

In [37]:
x_train, x_test, y_train, y_test = train_test_split(df[['age','affordability']], df.have_insurance, test_size=0.2, random_state=10)

In [38]:
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [39]:
x_test_scaled.shape

(6, 2)

In [40]:
x_train_scaled.shape

(22, 2)

In [41]:
x_train_scaled

,age,affordability
11,0.28,1
12,0.27,0
1,0.25,0
22,0.21,1
25,0.27,1
14,0.43,1
18,0.21,0
3,0.52,0
26,0.33,0
6,0.55,0


In [42]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,),activation='sigmoid', kernel_initializer='one', bias_initializer='zeros')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model.fit(x_train_scaled, y_train, epochs=5000)

Epoch 1/5000
1/1 [==============================] - 0s 214ms/step - loss: 0.7783 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7779 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7775 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7771 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7767 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.7763 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7758 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7754 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7750 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7746 - accuracy: 0.5000
Epoch 11/5000
1/1

In [44]:
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 0s 79ms/step - loss: 0.3750 - accuracy: 1.0000


[0.37496474385261536, 1.0]

In [47]:
predicts = model.predict(x_test_scaled)

1/1 [==============================] - 0s 16ms/step


In [48]:
predicts = [i for i in predicts]
predicts

[array([0.7497363], dtype=float32),
 array([0.6096149], dtype=float32),
 array([0.7133067], dtype=float32),
 array([0.6041684], dtype=float32),
 array([0.2545479], dtype=float32),
 array([0.28202203], dtype=float32)]

In [45]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [46]:
sigmoid(predicts[1])

TypeError: bad operand type for unary -: 'list'

In [49]:
predicts[1]

array([0.6096149], dtype=float32)

In [50]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.4292703 ],
        [0.03146307]], dtype=float32),
 array([-2.160356], dtype=float32))

In [51]:
y_test

7     1
21    1
5     1
2     1
13    0
19    0
Name: have_insurance, dtype: int64

In [52]:
x_test_scaled

,age,affordability
7,0.60,0
21,0.48,0
5,0.56,1
2,0.47,1
13,0.20,0
19,0.22,1


In [53]:
predicts

[[array([0.7497363], dtype=float32)],
 [array([0.6096149], dtype=float32)],
 [array([0.7133067], dtype=float32)],
 [array([0.6041684], dtype=float32)],
 [array([0.2545479], dtype=float32)],
 [array([0.28202203], dtype=float32)]]

# Only Python

In [53]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))

sigmoid(0)

0.5

In [54]:
def prediction_function(age, affordability):
    weighted_sum = coef[0]*age+coef[1]*affordability+intercept
    return sigmoid(weighted_sum)

In [55]:
prediction_function(.47,1)

0.6041684239020522

In [56]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [57]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [87]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshhold):
    w1 = w2 = 1
    bias = 0
    rate = 0.05
    n = len(age)
    
    for i in range(epochs):
        weighted_sum = w1*age+w2*affordability+bias
        y_predicted = sigmoid_numpy(weighted_sum)
        
        loss = log_loss(y_true, y_predicted)
        
        w1d = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability), (y_predicted-y_true))
        
        bias_d = np.mean(y_predicted-y_true)
        
        w1 = w1-rate*w1d
        w2 = w2-rate*w2d
        bias = bias-rate*bias_d
        
        print(f"Epoch: {i}, w1: {w1}, w2: {w2}, loss: {loss}")
        if loss<=loss_threshhold:
            break
    return w1,w2, bias

In [88]:
gradient_descent(x_train_scaled['age'], x_train_scaled['affordability'], y_train,10000, 0.5099)

Epoch: 0, w1: 0.998090084217315, w2: 0.9918161660608058, loss: 0.7782982719144392
Epoch: 1, w1: 0.9962373546803881, w2: 0.983716965962458, loss: 0.7747191255007618
Epoch: 2, w1: 0.9944414518460761, w2: 0.9757022537535253, loss: 0.7712319608050126
Epoch: 3, w1: 0.9927020022472104, w2: 0.9677718520450619, loss: 0.7678350590862713
Epoch: 4, w1: 0.9910186190002893, w2: 0.959925552699376, loss: 0.7645266973145298
Epoch: 5, w1: 0.989390902320911, w2: 0.9521631175542081, loss: 0.7613051501463816
Epoch: 6, w1: 0.9878184400455984, w2: 0.9444842791793245, loss: 0.7581686918458008
Epoch: 7, w1: 0.9863008081586909, w2: 0.9368887416625248, loss: 0.7551155981464709
Epoch: 8, w1: 0.9848375713230055, w2: 0.9293761814220715, loss: 0.7521441480524801
Epoch: 9, w1: 0.9834282834130008, w2: 0.9219462480425734, loss: 0.7492526255745904
Epoch: 10, w1: 0.982072488049214, w2: 0.9145985651313902, loss: 0.7464393213996531
Epoch: 11, w1: 0.9807697191327831, w2: 0.9073327311926748, loss: 0.743702534491113
Epoch: 1

(5.438037374724041, 0.009764815749284601, -2.143273955624412)

In [89]:
coef, intercept

(array([[5.4292703 ],
        [0.03146307]], dtype=float32),
 array([-2.160356], dtype=float32))